In [1]:
import ibis
import lonboard
import fiona
import geopandas as gpd

ibis.options.interactive = True

In [2]:
# Path to the GeoPackage file with multiple layers of geospatial data
geopackage_path = "input/gadm41_IND.gpkg"

In [3]:
# List all layer names in the GeoPackage using Fiona
layer_names = fiona.listlayers(geopackage_path)
layer_names

['ADM_ADM_0', 'ADM_ADM_1', 'ADM_ADM_2', 'ADM_ADM_3']

In [2]:
# Connect to DB
con = ibis.duckdb.connect('db/india_model.ddb', extensions = ['spatial'])

In [13]:
# Iterate over each layer in the GeoPackage
for layer_name in layer_names:
    # Read the current layer into a GeoDataFrame
    gdf = gpd.read_file(geopackage_path, layer=layer_name)

    # Set CRS to WGS84 (EPSG:4326) if it’s not already set
    if gdf.crs is None or gdf.crs != "EPSG:4326":
        gdf = gdf.to_crs("EPSG:4326")

    # Convert geometries to WKB format
    gdf = gdf.to_wkb()

    # Create an Ibis in-memory table from the GeoDataFrame data
    ibis_memtable = ibis.memtable(gdf)
    
    # Convert WKB geometry column to DuckDB geometry type for spatial operations
    ibis_memtable = ibis_memtable.mutate(geometry=ibis_memtable.geometry.cast("geometry"))

    # Create a table in DuckDB for the current layer, overwriting if it already exists
    con.create_table(layer_name, ibis_memtable, overwrite=True)

    # Clean up the in-memory table to free memory
    del ibis_memtable

In [6]:
con.list_tables()

['ADM_ADM_0',
 'ADM_ADM_1',
 'ADM_ADM_2',
 'ADM_ADM_3',
 'climate_data',
 'climate_data_with_point',
 'soil_water_layer1',
 'surface_solar_radiation_downwards',
 'temperature_2m',
 'total_precipitation',
 'wind_u_component_10m',
 'wind_v_component_10m']

In [3]:
con.table('ADM_ADM_3')

┏━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ GID_3       ┃ GID_0  ┃ COUNTRY ┃ GID_1   ┃ NAME_1              ┃ NL_NAME_1 ┃ GID_2     ┃ NAME_2                   ┃ NL_NAME_2 ┃ NAME_3       ┃ VARNAME_3 ┃ NL_NAME_3 ┃ TYPE_3 ┃ ENGTYPE_3 ┃ CC_3   ┃ HASC_3 ┃ geometry                                                                         ┃
┡━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ string      │ string │ string  │ string  │ string              │ string    │ string    │ string                   │ string    │ string       │ string    │ string    │ string │ string    │ string │ string │ geospatial:geometry                                                              │
├─────────────┼────────┼─────────┼─────────┼─────────────────────┼───────────┼───────────┼──────────────────────────┼───────────┼──────────────┼───────────┼───────────┼────────┼───────────┼────────┼────────┼──────────────────────────────────────────────────────────────────────────────────┤
│ IND.1.1.1_1 │ IND    │ India   │ IND.1_1 │ Andaman and Nicobar │ NA        │ IND.1.1_1 │ Nicobar Islands          │ NA        │ n.a. ( 2304) │ NA        │ NA        │ Taluk  │ Taluk     │ NA     │ NA     │ <MULTIPOLYGON (((93.79 6.852, 93.79 6.852, 93.791 6.852, 93.791 6.851, 93.79...> │
│ IND.1.2.1_1 │ IND    │ India   │ IND.1_1 │ Andaman and Nicobar │ NA        │ IND.1.2_1 │ North and Middle Andaman │ NA        │ n.a. ( 2178) │ NA        │ NA        │ Taluk  │ Taluk     │ NA     │ NA     │ <MULTIPOLYGON (((92.844 12.15, 92.845 12.15, 92.845 12.15, 92.845 12.15, 92....> │
│ IND.1.3.1_1 │ IND    │ India   │ IND.1_1 │ Andaman and Nicobar │ NA        │ IND.1.3_1 │ South Andaman            │ NA        │ n.a. ( 2178) │ NA        │ NA        │ Taluk  │ Taluk     │ NA     │ NA     │ <MULTIPOLYGON (((92.521 10.897, 92.523 10.895, 92.523 10.895, 92.526 10.891,...> │
│ IND.2.1.1_1 │ IND    │ India   │ IND.2_1 │ Andhra Pradesh      │ NA        │ IND.2.1_1 │ Anantapur                │ NA        │ Anantapur    │ NA        │ NA        │ Taluk  │ Taluk     │ NA     │ NA     │ <MULTIPOLYGON (((77.845 14.586, 77.839 14.588, 77.836 14.589, 77.83 14.591, ...> │
│ IND.2.1.2_1 │ IND    │ India   │ IND.2_1 │ Andhra Pradesh      │ NA        │ IND.2.1_1 │ Anantapur                │ NA        │ Dharmavaram  │ NA        │ NA        │ Taluk  │ Taluk     │ NA     │ NA     │ <MULTIPOLYGON (((77.355 14.522, 77.355 14.533, 77.375 14.536, 77.388 14.545,...> │
│ IND.2.1.3_1 │ IND    │ India   │ IND.2_1 │ Andhra Pradesh      │ NA        │ IND.2.1_1 │ Anantapur                │ NA        │ Gooty        │ NA        │ NA        │ Taluk  │ Taluk     │ NA     │ NA     │ <MULTIPOLYGON (((77.754 14.889, 77.742 14.893, 77.719 14.904, 77.714 14.91, ...> │
│ IND.2.1.4_1 │ IND    │ India   │ IND.2_1 │ Andhra Pradesh      │ NA        │ IND.2.1_1 │ Anantapur                │ NA        │ Hindupur     │ NA        │ NA        │ Taluk  │ Taluk     │ NA     │ NA     │ <MULTIPOLYGON (((77.904 14.024, 77.899 14.012, 77.899 14.006, 77.897 14.003,...> │
│ IND.2.1.5_1 │ IND    │ India   │ IND.2_1 │ Andhra Pradesh      │ NA        │ IND.2.1_1 │ Anantapur                │ NA        │ Kadiri       │ NA        │ NA        │ Taluk  │ Taluk     │ NA     │ NA     │ <MULTIPOLYGON (((77.86 14.35, 77.86 14.356, 77.857 14.364, 77.857 14.367, 77...> │
│ IND.2.1.6_1 │ IND    │ India   │ IND.2_1 │ Andhra Pradesh      │ NA        │ IND.2.1_1 │ Anantapur                │ NA        │ Kalyandurg   │ NA        │ NA        │ Taluk  │ Taluk     │ NA     │ NA     │ <MULTIPOL

In [20]:
lonboard.viz(
    con.table('ADM_ADM_1'), 
    polygon_kwargs={
        "get_fill_color": [255, 0, 0, 50],      # Semi-transparent red fill color
        "get_line_color": [0, 0, 0, 255],       # Black boundary lines
        "stroked": True,                        # Draw polygon boundaries
        "pickable": True,                       # Allow picking for interactivity
        "line_width_min_pixels": 1,             # Minimum width of the boundary lines
        "extruded": False,                      # Keep the layer 2D (no 3D extrusion)
        "auto_highlight": True,                 # Enable automatic highlighting on hover
    })

/home/denis/.cache/pypoetry/virtualenvs/india-climate-agri-database-dWLTZUkq-py3.12/lib/python3.12/site-packages/lonboard/_geoarrow/ops/reproject.py:33: UserWarning: No CRS exists on data. If no data is shown on the map, double check that your CRS is WGS84.
  warn(


Map(basemap_style=<CartoBasemap.DarkMatter: 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'…

In [19]:
con.table('ADM_ADM_3')

┏━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ GID_3       ┃ GID_0  ┃ COUNTRY ┃ GID_1   ┃ NAME_1              ┃ NL_NAME_1 ┃ GID_2     ┃ NAME_2                   ┃ NL_NAME_2 ┃ NAME_3       ┃ VARNAME_3 ┃ NL_NAME_3 ┃ TYPE_3 ┃ ENGTYPE_3 ┃ CC_3   ┃ HASC_3 ┃ geometry                                                                         ┃
┡━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ string      │ string │ string  │ string  │ string              │ string    │ string    │ string                   │ string    │ string       │ string    │ string    │ string │ string    │ string │ string │ geospatial:geometry                                                              │
├─────────────┼────────┼─────────┼─────────┼─────────────────────┼───────────┼───────────┼──────────────────────────┼───────────┼──────────────┼───────────┼───────────┼────────┼───────────┼────────┼────────┼──────────────────────────────────────────────────────────────────────────────────┤
│ IND.1.1.1_1 │ IND    │ India   │ IND.1_1 │ Andaman and Nicobar │ NA        │ IND.1.1_1 │ Nicobar Islands          │ NA        │ n.a. ( 2304) │ NA        │ NA        │ Taluk  │ Taluk     │ NA     │ NA     │ <MULTIPOLYGON (((93.79 6.852, 93.79 6.852, 93.791 6.852, 93.791 6.851, 93.79...> │
│ IND.1.2.1_1 │ IND    │ India   │ IND.1_1 │ Andaman and Nicobar │ NA        │ IND.1.2_1 │ North and Middle Andaman │ NA        │ n.a. ( 2178) │ NA        │ NA        │ Taluk  │ Taluk     │ NA     │ NA     │ <MULTIPOLYGON (((92.844 12.15, 92.845 12.15, 92.845 12.15, 92.845 12.15, 92....> │
│ IND.1.3.1_1 │ IND    │ India   │ IND.1_1 │ Andaman and Nicobar │ NA        │ IND.1.3_1 │ South Andaman            │ NA        │ n.a. ( 2178) │ NA        │ NA        │ Taluk  │ Taluk     │ NA     │ NA     │ <MULTIPOLYGON (((92.521 10.897, 92.523 10.895, 92.523 10.895, 92.526 10.891,...> │
│ IND.2.1.1_1 │ IND    │ India   │ IND.2_1 │ Andhra Pradesh      │ NA        │ IND.2.1_1 │ Anantapur                │ NA        │ Anantapur    │ NA        │ NA        │ Taluk  │ Taluk     │ NA     │ NA     │ <MULTIPOLYGON (((77.845 14.586, 77.839 14.588, 77.836 14.589, 77.83 14.591, ...> │
│ IND.2.1.2_1 │ IND    │ India   │ IND.2_1 │ Andhra Pradesh      │ NA        │ IND.2.1_1 │ Anantapur                │ NA        │ Dharmavaram  │ NA        │ NA        │ Taluk  │ Taluk     │ NA     │ NA     │ <MULTIPOLYGON (((77.355 14.522, 77.355 14.533, 77.375 14.536, 77.388 14.545,...> │
│ IND.2.1.3_1 │ IND    │ India   │ IND.2_1 │ Andhra Pradesh      │ NA        │ IND.2.1_1 │ Anantapur                │ NA        │ Gooty        │ NA        │ NA        │ Taluk  │ Taluk     │ NA     │ NA     │ <MULTIPOLYGON (((77.754 14.889, 77.742 14.893, 77.719 14.904, 77.714 14.91, ...> │
│ IND.2.1.4_1 │ IND    │ India   │ IND.2_1 │ Andhra Pradesh      │ NA        │ IND.2.1_1 │ Anantapur                │ NA        │ Hindupur     │ NA        │ NA        │ Taluk  │ Taluk     │ NA     │ NA     │ <MULTIPOLYGON (((77.904 14.024, 77.899 14.012, 77.899 14.006, 77.897 14.003,...> │
│ IND.2.1.5_1 │ IND    │ India   │ IND.2_1 │ Andhra Pradesh      │ NA        │ IND.2.1_1 │ Anantapur                │ NA        │ Kadiri       │ NA        │ NA        │ Taluk  │ Taluk     │ NA     │ NA     │ <MULTIPOLYGON (((77.86 14.35, 77.86 14.356, 77.857 14.364, 77.857 14.367, 77...> │
│ IND.2.1.6_1 │ IND    │ India   │ IND.2_1 │ Andhra Pradesh      │ NA        │ IND.2.1_1 │ Anantapur                │ NA        │ Kalyandurg   │ NA        │ NA        │ Taluk  │ Taluk     │ NA     │ NA     │ <MULTIPOL

#### Merge geometries

In [2]:
import duckdb

# Connect to DuckDB database
con = duckdb.connect('db/india_model.ddb')

# Load the spatial extension to enable spatial functions like ST_Contains
con.execute("INSTALL spatial; LOAD spatial;")

In [ ]:
result = con.execute("""
    PRAGMA ADM_ADM_3;
""").fetchdf()

CatalogException: Catalog Error: Pragma Function with name ADM_ADM_3 does not exist!
Did you mean "add_parquet_key"?